# 🤖 AI POWERED QUIZ APP

### Initialize model

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.3)

## Prompt

In [3]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system = """You are a Quiz Master AI. Your role is to generate engaging and informative quiz questions on any topic provided by the user. You should aim for a variety of question styles, including multiple-choice, true/false, and also include point on every question if user give correct answer to give that point to them ,and open-ended questions where appropriate. Ensure the questions are clear, concise, and factually accurate.
"""

example = [
    HumanMessage(
        content="Can you want me quiz on topic of programming. and should be in multiple-choice format."
    ),
    AIMessage(
        content="""
        Which keyword is used to define a function in Python? (7 points)
        a) func
        b) define
        c) def
        d) function
        """
    ),
    HumanMessage(
        content="So that answer is: c, In python we use def to create function."
    ),
    AIMessage(
        content="""{{"correct":True,"points":7}}"""
    ),
    AIMessage(
        content="""
        Which of the following is a valid variable name in Python? (9 points)

        a) 1st_variable
        b) first-variable
        c) first_variable
        d) first variable
        """
    ),
    HumanMessage(
        content="So that answer is: c."
    ),
    AIMessage(
        content="""{{"correct":False,"points":0}}"""
    ),
    AIMessage(
        content="""
        What is the output of the following code snippet? (5 points)
        ```python
        print(type([]) is list)
        ```
        a) True
        b) False
        c) None
        d) Error
        """
    ),
    HumanMessage(
        content="So that answer is: c."
    ),
    AIMessage(
        content="""{{"correct":False,"points":0}}"""
    ),
    AIMessage(
        content="""
        What is the time complexity of accessing an element in a list by index in Python? (10 points)
        a) O(1)
        b) O(n)
        c) O(log n)
        d) O(n^2)
        """
    ),
    HumanMessage(
        content="So that answer is: a."
    ),
    AIMessage(
        content="""{{"correct":True,"points":10}}"""
    ),
    HumanMessage(
        content="Can you want me quiz on topic of programming. But the format should be true/false."
    ),
    AIMessage(
        content="""
        True or False: Python is a statically typed language. (5 points)
        """
    ),
    HumanMessage(
        content="So that answer is: False."
    ),
    AIMessage(
        content="""{{"correct":True,"points":5}}"""
    ),
    AIMessage(
        content="""
        True or False: In Python, a tuple is mutable. (5 points)
        """
    ),
    HumanMessage(
        content="So that answer is: False."
    ),
    AIMessage(
        content="""{{"correct":False,"points":0}}"""
    ),
]

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=system),
        *example,
        MessagesPlaceholder("topic")
    ]
)

## Create a structure for output.

- Here we initialize the format of out in such case:
  - To create questions
  - To check answer.

In [12]:
from typing import List, Optional, Union
from pydantic import Field, BaseModel
from typing_extensions import Annotated


class QueryStructure(BaseModel):
    """Quiz for user."""
    question: Annotated[str, ..., "The quiz to ask user."]
    options: Optional[Annotated[List[str], ...,
                                "If quiz is multiple choice, the options of quiz"]]
    point: Annotated[int, ..., "The point of the quiz."]


class AnswerStructure(BaseModel):
    """Quiz answer check."""
    correct: Annotated[bool, ..., "The result of the quiz"]
    earn_point: Annotated[int, ..., "The point that user earn"]
    correct_answer: Annotated[str, ..., "The correct answer."]


class CombineStructure(BaseModel):
    combine_structure: Union[QueryStructure, AnswerStructure]


structured_llm = llm.with_structured_output(CombineStructure)

## Let's build a `Chain`.

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, chain


points = 0
earn_points = 0

messages = []
history_chat = []


def print_quiz(quiz) -> AIMessage:
    global points

    quiz_data = quiz.combine_structure
    # Printing quiz
    print(f"Question: {quiz_data.question}")
    if quiz_data.options is not None:
        for i, option in enumerate(quiz_data.options):
            print(f"{chr(97 + i)}) {option}")

    points += quiz_data.point
    return AIMessage(content=str(quiz_data))


def getAnswerFromUser() -> HumanMessage:
    """
    Get answer from the user.
    """
    user_input = input("Enter you answer:- ")
    answer = user_input if user_input else "I don't know."

    human_message = HumanMessage(content=answer)
    return human_message


def increase_points(result):
    global earn_points

    quiz_result = result.combine_structure
    if "correct" in quiz_result and "earn_points" in quiz_result:
        earn_points += quiz_result.earn_point
        message = "Your answer is correct" if quiz_result.correct else "Sorry, Your answer is wrong."

        print(message)


@chain
def quiz_executor(topic):
    llm_with_ps = prompt | structured_llm

    quiz = llm_with_ps.invoke({"topic": [HumanMessage(content=topic)]})

    ai_message = print_quiz(quiz)
    human_message = getAnswerFromUser()
    combine_message = [ai_message, human_message]

    quiz_result = llm_with_ps.invoke({"topic": combine_message})
    return quiz_result


quiz_executor.invoke("on the topic of indian history with multiple choice.")

Question: Who was the first Prime Minister of India?
a) Jawaharlal Nehru
b) Indira Gandhi
c) Mahatma Gandhi
d) Subhas Chandra Bose


CombineStructure(combine_structure=QueryStructure(question='Who was the first Prime Minister of India?', options=['Jawaharlal Nehru', 'Indira Gandhi', 'Mahatma Gandhi', 'Subhas Chandra Bose'], point=10))

In [9]:
topic = None

while True:
    messages = []

    if (len(history_chat) > 0):
        isExit = input("Do you want to continues you exit. (y/n)")
        if isExit == "n" or isExit == "no":
            if points > 0:
                print(f"You earn {earn_points} out of {points}")
            break

    if topic is None:
        user_input = input("Enter you topic name with format:- ")
        topic = user_input if user_input else "random topic any format."

    quiz_result = quiz_executor.invoke(topic)
    # increase_points(quiz_result)
    print(quiz_result)
    break

Question: Who was the first Prime Minister of India?
a) Jawaharlal Nehru
b) Mahatma Gandhi
c) Subhas Chandra Bose
d) Lal Bahadur Shastri
combine_structure=QueryStructure(question='Who was the first Prime Minister of India?', options=['Jawaharlal Nehru', 'Mahatma Gandhi', 'Subhas Chandra Bose', 'Lal Bahadur Shastri'], point=5)


In [ ]:
print(structured_llm.invoke([AIMessage(content="question='Who was the last Governor-General of independent India before the country became a republic?' options=['C. Rajagopalachari', 'Lord Mountbatten', 'Lord Wavell', 'Lord Linlithgow'] point=8",
      additional_kwargs={}, response_metadata={}), HumanMessage(content='So my answer is: b', additional_kwargs={}, response_metadata={})]))

combine_structure=AnswerStructure(correct=False, earn_point=0, correct_answer='C. Rajagopalachari')
